# REFINITIV WACC MODEL

### Instructions

Please input a valid RIC after running the first and second cell below. Then you can run the rest of the notebook using shift+enter for each cell sequentially. This notebook will provide all of the information contained in the popular Eikon/Workspace template of the same name. This notebook shows how StarMine WACC and StarMine ERP (Equity Risk Premium - a component of StarMine WACC) are calculated. 

In the Manual Recalculation cell, you can overwrite variables to calculate WACC based on the StarMine methodologies but using your own estimates and other inputs.

Note StarMine ERP covers 40 countries only, but StarMine WACC includes stocks listed in non-supported countries, e.g. JMBN.BEL in Serbia.  In these instances, you can enter your own value for ERP in the Manual Recalculation, but you cannot recalculate ERP using the StarMine ERP model.

### Imports

In [2]:
import refinitiv.data as rd
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Box, Label, Layout
import matplotlib as plt
import warnings
%matplotlib inline
plt.style.use("seaborn")
warnings.filterwarnings("ignore")
rd.open_session()

<refinitiv.data.session.Definition object at 0x16b2df070 {name='workspace'}>

### Select Equity RIC

In [3]:
ric = widgets.Text(
    value='',
    placeholder='Type something',
    description='RIC:',
    disabled=False
)

box_layout = Layout(display='flex',
                    flex_flow='row wrap',
                    align_items='stretch',
                    border='solid',
                    width='100%')

items = [ric]
box = Box(children=items, layout=box_layout)

box

Box(children=(Text(value='', description='RIC:', placeholder='Type something'),), layout=Layout(align_items='s…

### Get WACC data

In [4]:
wacc1 = rd.get_data(ric.value, ['TR.WACC', 'TR.WACCCostofDebt', 'TR.WACCCostofEquity',
                                'TR.WACCCostofPreferred', 'TR.WACCERP', 'TR.WACCInflationAdjRiskFreeRate',
                                'TR.WACCDebtWeight', 'TR.WACCEquityWeight', 'TR.WACCPreferredWeight',
                                'TR.WACCTaxRate', 'TR.WACCShortTermDebtCost', 'TR.WACCLongTermDebtCost',
                                'TR.WACCBeta', 'TR.WACCCompanyTaxRateMedian'])
wacc1.transpose()

,0
Instrument,VOD.L
"Weighted Average Cost of Capital, (%)",6.619253
"WACC Cost of Debt, (%)",4.9991
"WACC Cost of Equity, (%)",10.459205
"WACC Cost of Preferred, (%)",0
"WACC Equity Risk Premium, (%)",7.139906
"WACC Inflation Adjusted Risk Free Rate, (%)",3.728314
"WACC Debt Weight, (%)",70.327442
"WACC Equity Weight, (%)",29.672558
"WACC Preferred Weight, (%)",0


In [5]:
wacc2 = rd.get_data(ric.value, ["100-(TR.WACCCostofDebt/(1-TR.WACCTaxRate/100)-TR.WACCShortTermDebtCost)/(TR.WACCLongTermDebtCost-TR.WACCShortTermDebtCost)*100",
                              "(TR.WACCCostofDebt/(1-TR.WACCTaxRate/100)-TR.WACCShortTermDebtCost)/(TR.WACCLongTermDebtCost-TR.WACCShortTermDebtCost)*100"])

wacc2.columns = [
    'Instrument', 'Implied Weight of Short Term Debt (%)', 'Implied Weight of Long Term Debt (%)']
if wacc1['WACC Company Tax Rate Median, (%)'][0] == wacc1['WACC Tax Rate, (%)'][0]:
    wacc2['Unstable Tax Rate Adjustment Applied'] = 'No'
else:
    wacc2['Unstable Tax Rate Adjustment Applied'] = 'Yes'
wacc2.transpose()

,0
Instrument,VOD.L
Implied Weight of Short Term Debt (%),20.739065
Implied Weight of Long Term Debt (%),79.260935
Unstable Tax Rate Adjustment Applied,Yes


In [6]:
ISO2 = rd.get_data(ric.value, 'TR.HQAddressCountryISO')
smRIC = ISO2['HQ Address Country ISO'][0] + 'GVERP=SM'
wacc3 = rd.get_data(smRIC, ['DSPLY_NAME', 'GN_TXT24_1', 'HST_CLOSE', 'GEN_VAL1', 'GEN_VAL2',
                            'GEN_VAL3', 'GEN_VAL4', 'GEN_VAL5', 'GEN_VAL6', 'GEN_VAL7',
                            'RT_YIELD_1', 'GEN_VAL8'])

wacc3.columns = ['Instrument', 'Country', 'Index', 'ERP', 'Expected Market Return',
                 'Inflation Adj. Risk Free Rate', 'Implied Dividend Payout Ratio',
                 'FY1 Earnings Yield', 'FY1 Return on Equity', 'FY10 Forecast GDP Growth',
                 'FY10 Forecast Inflation', '10Yr Gov Bond Yield', '10Yr Inflation Forecast']
wacc3.T

,0
Instrument,GBGVERP=SM
Country,UNITED KINGDOM
Index,FT100
ERP,7.17
Expected Market Return,10.88
Inflation Adj. Risk Free Rate,3.7
Implied Dividend Payout Ratio,76.5
FY1 Earnings Yield,9.4
FY1 Return on Equity,15.69
FY10 Forecast GDP Growth,3.69


### Manual Recalculation (WACC) - please change values below and hit recalculate button

Here we use ipywidgets to render a basic UI cell where the user can change various values and recalculate Final WACC %. 

In [7]:
waccTitle = widgets.Label(
    'Enter Values and Recalc WACC',
    layout=Layout(width='650px')
    # disabled=True,
    #style= {'description_width': 'initial'},
    # layout=Layout(width='650px')
)
waccCPREF = widgets.Text(
    value=str(wacc1['WACC Cost of Preferred, (%)'][0]),
    description='WACC Cost of Preferred, (%):',
    disabled=False,
    style={'description_width': 'initial'},
    layout=Layout(width='650px')
)
waccERP = widgets.Text(
    value=str(wacc1['WACC Equity Risk Premium, (%)'][0]),
    description='WACC Equity Risk Premium, (%):',
    disabled=False,
    style={'description_width': 'initial'},
    layout=Layout(width='650px')
)
waccIARFR = widgets.Text(
    value=str(wacc1['WACC Inflation Adjusted Risk Free Rate, (%)'][0]),
    description='WACC Inflation Adjusted Risk Free Rate, (%):',
    disabled=False,
    style={'description_width': 'initial'},
    layout=Layout(width='650px')
)
waccDW = widgets.Text(
    value=str(wacc1['WACC Debt Weight, (%)'][0]),
    description='WACC Debt Weight, (%):',
    disabled=False,
    style={'description_width': 'initial'},
    layout=Layout(width='650px')
)
waccEW = widgets.Text(
    value=str(wacc1['WACC Equity Weight, (%)'][0]),
    description='WACC Equity Weight, (%):',
    disabled=False,
    style={'description_width': 'initial'},
    layout=Layout(width='650px')
)
waccPW = widgets.Text(
    value=str(wacc1['WACC Preferred Weight, (%)'][0]),
    description='WACC Preferred Weight, (%):',
    disabled=False,
    style={'description_width': 'initial'},
    layout=Layout(width='650px')
)
waccTR = widgets.Text(
    value=str(wacc1['WACC Tax Rate, (%)'][0]),
    description='WACC Tax Rate, (%):',
    disabled=False,
    style={'description_width': 'initial'},
    layout=Layout(width='650px')
)
waccSTDC = widgets.Text(
    value=str(wacc1['WACC Short Term Debt Cost, (%)'][0]),
    description='WACC Short Term Debt Cost, (%):',
    disabled=False,
    style={'description_width': 'initial'},
    layout=Layout(width='650px')
)
waccLTDC = widgets.Text(
    value=str(wacc1['WACC Long Term Debt Cost, (%)'][0]),
    description='WACC Long Term Debt Cost, (%):',
    disabled=False,
    style={'description_width': 'initial'},
    layout=Layout(width='650px')
)
waccBETA = widgets.Text(
    value=str(wacc1['Beta'][0]),
    description='Beta:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=Layout(width='650px')
)
waccCTRM = widgets.Text(
    value=str(wacc1['WACC Company Tax Rate Median, (%)'][0]),
    description='WACC Company Tax Rate Median, (%):',
    disabled=False,
    style={'description_width': 'initial'},
    layout=Layout(width='650px')
)
waccSTDW = widgets.Text(
    value=str(wacc2['Implied Weight of Short Term Debt (%)'][0]),
    description='Implied Weight of Short Term Debt (%):',
    disabled=False,
    style={'description_width': 'initial'},
    layout=Layout(width='650px')
)
waccLTDW = widgets.Text(
    value=str(wacc2['Implied Weight of Long Term Debt (%)'][0]),
    description='Implied Weight of Long Term Debt (%):',
    disabled=False,
    style={'description_width': 'initial'},
    layout=Layout(width='650px')
)
KE = float(waccIARFR.value) + float(waccBETA.value) * float(waccERP.value)
KD = (float(waccSTDC.value) * (float(waccSTDW.value)/100) + float(waccLTDC.value)
      * float(waccLTDW.value)/100) * (1-(float(waccTR.value)/100))
WACC = KE * (float(waccEW.value)/100) + KD * (float(waccDW.value) /
                                              100) + float(waccCPREF.value) * (float(waccPW.value)/100)
waccFinal = widgets.Text(
    value=str(WACC),
    description='Final WACC (%):',
    disabled=True,
    style={'description_width': 'initial'},
    layout=Layout(width='650px')
)
waccBUT = widgets.Button(
    description='Recalculate',
    disabled=False,
    display='flex',
    flex_flow='column',
    align_items='stretch',
    layout=Layout(width='650px')
)


def recalc(self):
    try:
        KE = float(waccIARFR.value) + \
            float(waccBETA.value) * float(waccERP.value)
        KD = (float(waccSTDC.value) * (float(waccSTDW.value)/100) + float(waccLTDC.value)
              * float(waccLTDW.value)/100) * (1-(float(waccTR.value)/100))
        WACC = KE * (float(waccEW.value)/100) + KD * (float(waccDW.value) /
                                                      100) + float(waccCPREF.value) * (float(waccPW.value)/100)
        waccFinal.value = str(WACC)
    except:
        print('Cannot Compute - please debug calculation')
        pass


waccBUT.on_click(recalc)

box_layout2 = Layout(display='flex',
                     flex_flow='column',
                     align_items='stretch',
                     border='solid',
                     width='700px')

items2 = [waccTitle, waccCPREF, waccERP, waccIARFR, waccDW, waccEW, waccPW, waccTR, waccSTDC, waccLTDC,
          waccBETA, waccCTRM, waccFinal, waccBUT]
box2 = Box(children=items2, layout=box_layout2)

box2

Box(children=(Label(value='Enter Values and Recalc WACC', layout=Layout(width='650px')), Text(value='0', descr…

In [8]:
rd.close_session()

**Further Resources for Refinitiv Data Libraries API on Developer Community Portal**

* [Overview](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python) 
* [Quick Start ](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start)
* [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/documentation)
* [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/tutorials)
* [Q&A Forums](https://community.developers.refinitiv.com/spaces/321/index.html)